# Stripe Connect: On-Demand Delivery Platform

**A DoorDash-style marketplace demonstrating Stripe Connect fundamentals**

---

### Quick Navigation
| Section | Topic |
|---------|-------|
| [**1. Onboarding**](#section-1-onboarding-connected-accounts) | Create restaurant & courier accounts |
| [**2. Payment**](#section-2-collecting-customer-payment) | Customer pays $35 for an order |
| [**3. Transfers**](#section-3-routing-funds-with-transfers) | Split payment to multiple parties |
| [**4. Edge Cases**](#section-4-edge-cases--real-world-scenarios) | Refunds, tips, instant payouts |

---

### The Players

| Party | Role | Stripe Representation |
|-------|------|----------------------|
| **Platform** | Delivery service (us) | Main Stripe account |
| **Restaurant** | Provides food | Custom Connected Account |
| **Courier** | Delivers food | Custom Connected Account |
| **Customer** | Pays for order | Customer object |

### Money Flow ($35 order)

```
Customer pays $35.00
    │
    ▼
Platform Stripe Balance ($35.00)
    │
    ├── Transfer $25.00 ──► Restaurant (food cost)
    ├── Transfer $7.00  ──► Courier (delivery fee)
    └── Keep $3.00      ──► Platform (service fee)
```

---
# Setup

In [80]:
import stripe
from stripe import StripeClient
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../.env')

# Initialize Stripe with secret key
stripe_api_key = os.getenv('STRIPE_SECRET_KEY')
if stripe_api_key is None:
    raise ValueError("STRIPE_SECRET_KEY not found in environment variables. Please check your .env file.")

# Set up both module-level API and StripeClient
stripe.api_key = stripe_api_key
client = StripeClient(stripe_api_key)

# Verify connection using the module-level API (more reliable for account retrieval)
account = stripe.Account.retrieve()
print(f"Connected to Stripe account: {account.id}")
display_name = account.settings.dashboard.display_name if account.settings and account.settings.dashboard else None
print(f"Business name: {display_name or 'Not set'}")

Connected to Stripe account: acct_1Smc08JCEv6vweOQ
Business name: SA Interview


---
# Section 1: Onboarding Connected Accounts

> **Goal:** Create Custom connected accounts for a restaurant and courier

### Why Custom Accounts?

For a delivery platform, we use **Custom** connected accounts because:
- **Full control** over the user experience (white-label onboarding)
- **Platform manages** communication with connected accounts
- **Flexible payout** timing and control
- Best for platforms where users don't need direct Stripe Dashboard access

### Onboarding Flow

```
Create Account → Collect KYC Info → Accept ToS → Verification → Enable Payouts
```

### 1.1 Create Restaurant Account

In [81]:
# Create a connected account for a restaurant using v2 API
restaurant_account = client.v2.core.accounts.create(
    params={
        "contact_email": "restaurant@example.com",
        "display_name": "Mario's Pizza",
        "dashboard": "none",  # Connected account gets dashboard access
        "identity": {
            "country": "US",
            "entity_type": "company",
            "business_details": {
                "registered_name": "Mario's Pizza",
            },
            # Note: ToS acceptance is handled by account owner via onboarding flow
            # when dashboard="express" and losses_collector="application"
        },
        "defaults": {
            "responsibilities": {"losses_collector": "application", "fees_collector": "application"},
            "currency": "usd",
        },
        "configuration": {
            # Only recipient config needed - restaurant receives transfers, doesn't process payments directly
            "recipient": {
                "capabilities": {"stripe_balance": {"stripe_transfers": {"requested": True}}},
            },
        },
        "metadata": {
            "platform_role": "restaurant",
            "internal_id": "rest_001",
        },
    }
)

print(f"Restaurant Account ID: {restaurant_account.id}")
print(f"Dashboard: {restaurant_account.dashboard}")
print(f"Display Name: {restaurant_account.display_name}")

Restaurant Account ID: acct_1Smg1nJCEvp7tW2L
Dashboard: none
Display Name: Mario's Pizza


### 1.2 Check Restaurant Account Requirements

Before an account can receive transfers, Stripe requires certain information for compliance:

In [84]:
# Check what's needed for the restaurant account
account = client.v1.accounts.retrieve(restaurant_account.id)

print("=== Restaurant Account Requirements ===")

if account.requirements:
    currently_due = account.requirements.currently_due or []
    eventually_due = account.requirements.eventually_due or []
    pending_verification = account.requirements.pending_verification or []
    
    print(f"\nCurrently due: {currently_due if currently_due else 'None'}")
    print(f"\nEventually due: {eventually_due if eventually_due else 'None'}")
    print(f"\nPending verification: {pending_verification if pending_verification else 'None'}")
else:
    print("\nNo requirements information available.")

=== Restaurant Account Requirements ===

Currently due: None

Eventually due: None

Pending verification: None


### 1.3 Update Business Details, Accept ToS & Add Bank Account

After account creation, update business information, accept the Terms of Service, and add an external bank account for payouts:

In [83]:
# Update business details and accept ToS for the restaurant account using v1 API
import time

updated_account = client.v1.accounts.update(
    restaurant_account.id,
    params={
        "business_type": "company",
        "company": {
            "name": "Mario's Pizza LLC",
            "address": {
                "line1": "123 Main St",
                "city": "San Francisco",
                "state": "CA",
                "postal_code": "94102",
                "country": "US",
            },
            "phone": "4155551234",
            "tax_id": "000000000",
        },
        "business_profile": {
            "url": "https://mariospizza.com",
            "mcc": "5812",  # Restaurants
        },
        # Accept Terms of Service on behalf of the connected account
        "tos_acceptance": {
            "date": int(time.time()),
            "ip": "8.8.8.8",  # IP address of the account holder accepting ToS
        },
    },
)

print(f"Updated Account ID: {updated_account.id}")
print(f"Business Type: {updated_account.business_type}")
if updated_account.tos_acceptance:
    print(f"ToS Accepted: {updated_account.tos_acceptance.date is not None}")

# Create external bank account for payouts
# Using test routing/account numbers: https://docs.stripe.com/connect/testing
restaurant_bank_account = stripe.Account.create_external_account(
    restaurant_account.id,
    external_account={
        "object": "bank_account",
        "country": "US",
        "currency": "usd",
        "routing_number": "110000000",  # Test routing number
        "account_number": "000123456789",  # Test account number
        "account_holder_name": "Mario's Pizza LLC",
        "account_holder_type": "company",
    },
)

print(f"\nBank Account ID: {restaurant_bank_account.id}")
print(f"Bank Name: {restaurant_bank_account.bank_name}")
print(f"Last 4: {restaurant_bank_account.last4}")
print(f"Default for currency: {restaurant_bank_account.default_for_currency}")

Updated Account ID: acct_1Smg1nJCEvp7tW2L
Business Type: company
ToS Accepted: True

Bank Account ID: ba_1Smg26JCEvp7tW2Lp1IoCv8n
Bank Name: STRIPE TEST BANK
Last 4: 6789
Default for currency: True


### 1.4 Create Courier Account

In [85]:
import time

# Create a connected account for a courier (individual) using v1 API
courier_account = client.v1.accounts.create(
    params={
        "country": "US",
        "type": "custom",
        "email": "courier@example.com",
        "business_type": "individual",
        "individual": {
            "first_name": "Alex",
            "last_name": "Johnson",
            "phone": "5559876543",
            "dob": {"day": 15, "month": 6, "year": 1990},
            "address": {
                "line1": "456 Oak Ave",
                "city": "San Francisco",
                "state": "CA",
                "postal_code": "94102",
                "country": "US",
            },
            "ssn_last_4": "0000",
        },
        "capabilities": {
            "transfers": {"requested": True},
        },
        "business_profile": {
            "url": "https://accessible.stripe.com",
        },
        "metadata": {
            "platform_role": "courier",
            "internal_id": "cour_001",
        },
        # Accept Terms of Service on behalf of the connected account
        "tos_acceptance": {
            "date": int(time.time()),
            "ip": "8.8.8.8",  # IP address of the account holder accepting ToS
        },
    }
)

print(f"Courier Account ID: {courier_account.id}")
print(f"Business Type: {courier_account.business_type}")
print(f"Email: {courier_account.email}")

Courier Account ID: acct_1Smg2jByLTCTtseH
Business Type: individual
Email: courier@example.com


### 1.5 Check Courier Account Requirements

Before an account can receive transfers, Stripe requires certain information for compliance:

In [88]:
# Check what's needed for the courier account
account = client.v1.accounts.retrieve(courier_account.id)

print("=== Courier Account Requirements ===")

if account.requirements:
    currently_due = account.requirements.currently_due or []
    eventually_due = account.requirements.eventually_due or []
    pending_verification = account.requirements.pending_verification or []
    
    print(f"\nCurrently due: {currently_due if currently_due else 'None'}")
    print(f"\nEventually due: {eventually_due if eventually_due else 'None'}")
    print(f"\nPending verification: {pending_verification if pending_verification else 'None'}")
else:
    print("\nNo requirements information available.")

=== Courier Account Requirements ===

Currently due: None

Eventually due: None

Pending verification: None


### 1.6 Add Bank Account for Courier

Add an external bank account for payouts:

In [87]:
# Create external bank account for payouts
# Using test routing/account numbers: https://docs.stripe.com/connect/testing
courier_bank_account = stripe.Account.create_external_account(
    courier_account.id,
    external_account={
        "object": "bank_account",
        "country": "US",
        "currency": "usd",
        "routing_number": "110000000",  # Test routing number
        "account_number": "000123456789",  # Test account number
        "account_holder_name": "Alex Johnson",
        "account_holder_type": "individual",
    },
)

print(f"\nBank Account ID: {courier_bank_account.id}")
print(f"Bank Name: {courier_bank_account.bank_name}")
print(f"Last 4: {courier_bank_account.last4}")
print(f"Default for currency: {courier_bank_account.default_for_currency}")


Bank Account ID: ba_1Smg2wByLTCTtseHk2ce2eWS
Bank Name: STRIPE TEST BANK
Last 4: 6789
Default for currency: True


---
# Section 2: Collecting Customer Payment

> **Goal:** Create a $35 order and collect payment from the customer

### Payment Flow

```
Create Customer → Create PaymentIntent ($35) → Confirm with Payment Method
```

We're using **Separate Charges and Transfers**:
- Payment goes to platform's Stripe balance first
- Platform then transfers funds to connected accounts
- Gives platform full control over fund routing and timing

### 2.1 Create Customer

In [89]:
# Create a customer for the order
customer = client.v1.customers.create(
    params={
        "email": "hungry.customer@example.com",
        "name": "Jane Smith",
        "metadata": {
            "platform_user_id": "user_12345"
        }
    }
)

print(f"Customer ID: {customer.id}")
print(f"Customer email: {customer.email}")

Customer ID: cus_TkAOJkJz7wkdIr
Customer email: hungry.customer@example.com


### 2.2 Create PaymentIntent

**Order breakdown:**
| Item | Amount |
|------|--------|
| Food (subtotal) | $25.00 |
| Delivery fee | $7.00 |
| Platform fee | $3.00 |
| **Total** | **$35.00** |

In [90]:
# Order amounts (in cents)
FOOD_COST = 2500      # $25.00 - goes to restaurant
DELIVERY_FEE = 700    # $7.00 - goes to courier
PLATFORM_FEE = 300    # $3.00 - platform keeps
TOTAL_AMOUNT = FOOD_COST + DELIVERY_FEE + PLATFORM_FEE  # $35.00

# Create PaymentIntent
payment_intent = client.v1.payment_intents.create(
    params={
        "amount": TOTAL_AMOUNT,
        "currency": "usd",
        "customer": customer.id,
        "metadata": {
            "order_id": "order_98765",
            "restaurant_id": restaurant_account.id,
            "courier_id": courier_account.id,
            "food_cost": str(FOOD_COST),
            "delivery_fee": str(DELIVERY_FEE),
            "platform_fee": str(PLATFORM_FEE),
        },
        "description": "Order #98765 - Mario's Pizza",
    }
)

print(f"PaymentIntent ID: {payment_intent.id}")
print(f"Amount: ${payment_intent.amount / 100:.2f}")
print(f"Status: {payment_intent.status}")
if payment_intent.client_secret:
    print(f"Client Secret: {payment_intent.client_secret[:30]}...")

PaymentIntent ID: pi_3Smg42JCEv6vweOQ1cRMIxo2
Amount: $35.00
Status: requires_payment_method
Client Secret: pi_3Smg42JCEv6vweOQ1cRMIxo2_se...


### 2.3 Confirm Payment

In production, the client uses Stripe.js/Elements. Here we simulate with a test card:

In [91]:
# Confirm payment using test payment method
# For testing/demo purposes, we'll use a direct confirmation approach
try:
    confirmed_payment = client.v1.payment_intents.confirm(
        payment_intent.id,
        params={
            "payment_method": "pm_card_visa",  # Use test payment method directly
            "return_url": "https://yourplatform.com/payment/return",  # Required for some payment methods
        }
    )
    
    print(f"Payment Status: {confirmed_payment.status}")
    print(f"Amount captured: ${confirmed_payment.amount_received / 100:.2f}")

    if confirmed_payment.status == "succeeded":
        print("\n Payment successful! Funds are now in platform's Stripe balance.")
        print("  Ready to transfer to restaurant and courier.")
        
except stripe.InvalidRequestError as e:
    print("Warning: Payment confirmation failed.")
    print("This is common in test environments. In production, you'd use:")
    print("1. Stripe Elements/Checkout for secure card collection")
    print("2. Client-side confirmation with proper return URLs")
    print(f"\nError: {str(e)[:100]}...")
    
    # Create mock successful payment for demo continuation
    confirmed_payment = type('MockPayment', (), {
        'status': 'succeeded',
        'amount_received': TOTAL_AMOUNT,
        'latest_charge': 'ch_mock_charge_12345'
    })()
    
    print(f"\nUsing mock payment for demo: {confirmed_payment.status}")
    print(f"   Mock charge ID: {confirmed_payment.latest_charge}")
    print("   Ready to transfer to restaurant and courier.")

Payment Status: succeeded
Amount captured: $35.00

 Payment successful! Funds are now in platform's Stripe balance.
  Ready to transfer to restaurant and courier.


---
# Section 3: Routing Funds with Transfers

> **Goal:** Split the $35 payment between restaurant, courier, and platform

### Why Separate Charges & Transfers?

- **Timing control** — Transfer after delivery confirmed, not before
- **Amount control** — Adjust for tips, refunds, promotions
- **Multi-party** — Split to any number of connected accounts
- **Dispute handling** — Platform holds funds, manages disputes centrally

### 3.1 Transfer to Restaurant ($25)

In [92]:
# Get the charge ID from the successful PaymentIntent
charge_id = confirmed_payment.latest_charge
print(f"Source Charge ID: {charge_id}")

# Transfer food cost to restaurant
try:
    restaurant_transfer = client.v1.transfers.create(
        params={
            "amount": FOOD_COST,  # $25.00
            "currency": "usd",
            "destination": restaurant_account.id,
            "source_transaction": charge_id,  # Links transfer to original charge
            "metadata": {
                "order_id": "order_98765",
                "transfer_type": "food_payment",
            },
            "description": "Order #98765 - Food payment",
        }
    )

    print(f"\nRestaurant Transfer ID: {restaurant_transfer.id}")
    print(f"Amount: ${restaurant_transfer.amount / 100:.2f}")
    print(f"Destination: {restaurant_transfer.destination}")
    
except stripe.InvalidRequestError as e:
    if "capabilities enabled" in str(e):
        print("\nWarning: Connected Account Not Ready for Transfers")
        print("=" * 45)
        print("The restaurant account needs to complete onboarding first:")
        print("1. Complete the Account Link onboarding flow")
        print("2. Provide all required business information")
        print("3. Wait for Stripe to verify and enable capabilities")
        print("\nIn production, you'd check account.capabilities.transfers.status")
        
        # Create mock transfer for demo
        restaurant_transfer = type('MockTransfer', (), {
            'id': 'tr_mock_restaurant_12345',
            'amount': FOOD_COST,
            'destination': restaurant_account.id
        })()
        
        print(f"\nUsing mock transfer: {restaurant_transfer.id}")
        print(f"Amount: ${restaurant_transfer.amount / 100:.2f}")
        print(f"Destination: {restaurant_transfer.destination}")
    else:
        raise

Source Charge ID: ch_3Smg42JCEv6vweOQ1ZyBMI7y

Restaurant Transfer ID: tr_3Smg42JCEv6vweOQ1C1K2VUM
Amount: $25.00
Destination: acct_1Smg1nJCEvp7tW2L


### 3.2 Transfer to Courier ($7)

In [93]:
# Transfer delivery fee to courier
try:
    courier_transfer = client.v1.transfers.create(
        params={
            "amount": DELIVERY_FEE,  # $7.00
            "currency": "usd",
            "destination": courier_account.id,
            "source_transaction": charge_id,
            "metadata": {
                "order_id": "order_98765",
                "transfer_type": "delivery_fee",
            },
            "description": "Order #98765 - Delivery fee",
        }
    )

    print(f"Courier Transfer ID: {courier_transfer.id}")
    print(f"Amount: ${courier_transfer.amount / 100:.2f}")
    print(f"Destination: {courier_transfer.destination}")

except stripe.InvalidRequestError as e:
    if "capabilities enabled" in str(e):
        print("\nWarning: Courier Account Not Ready for Transfers")
        print("Similar to restaurant - needs onboarding completion")
        
        # Create mock transfer for demo
        courier_transfer = type('MockTransfer', (), {
            'id': 'tr_mock_courier_12345',
            'amount': DELIVERY_FEE,
            'destination': courier_account.id
        })()
        
        print(f"\nUsing mock transfer: {courier_transfer.id}")
        print(f"Amount: ${courier_transfer.amount / 100:.2f}")
        print(f"Destination: {courier_transfer.destination}")
    else:
        raise

Courier Transfer ID: tr_3Smg42JCEv6vweOQ1F4tQVLS
Amount: $7.00
Destination: acct_1Smg2jByLTCTtseH


### 3.3 Verify Fund Distribution

In [94]:
# Summary of fund distribution
print("=" * 50)
print("FUND DISTRIBUTION SUMMARY")
print("=" * 50)
print(f"\nOrder Total:           ${TOTAL_AMOUNT / 100:.2f}")
print(f"\n  -> Restaurant:        ${FOOD_COST / 100:.2f}")
print(f"  -> Courier:           ${DELIVERY_FEE / 100:.2f}")
print(f"  -> Platform (kept):   ${PLATFORM_FEE / 100:.2f}")
print(f"\n  Total distributed:   ${(FOOD_COST + DELIVERY_FEE + PLATFORM_FEE) / 100:.2f}")
print("=" * 50)

# Verify with transfer list
transfers = client.v1.transfers.list(params={"limit": 2})
print("\nRecent Transfers:")
for t in transfers.data:
    print(f"  {t.id}: ${t.amount/100:.2f} -> {t.destination}")

FUND DISTRIBUTION SUMMARY

Order Total:           $35.00

  -> Restaurant:        $25.00
  -> Courier:           $7.00
  -> Platform (kept):   $3.00

  Total distributed:   $35.00

Recent Transfers:
  tr_3Smg42JCEv6vweOQ1F4tQVLS: $7.00 -> acct_1Smg2jByLTCTtseH
  tr_3Smg42JCEv6vweOQ1C1K2VUM: $25.00 -> acct_1Smg1nJCEvp7tW2L


---
# Section 4: Edge Cases & Real-World Scenarios

> **Goal:** Handle common platform scenarios like refunds, tips, and instant payouts

### 4.1 Refunds

In [75]:
# Full refund example (creates a new payment to demonstrate)
# In production, you'd refund the actual order's payment

# Create a refund
refund = stripe.Refund.create(
    charge=charge_id,
    # For partial refund, specify amount:
    # amount=1500,  # $15.00
)

print("Refund Considerations:")
print("- Full refund: Reverse all transfers, refund customer")
print("- Partial refund: May need to adjust transfers proportionally")
print("- Transfer reversals: Use stripe.Transfer.create_reversal()")
print("- Timing: Refunds can take 5-10 business days to appear")

Refund Considerations:
- Full refund: Reverse all transfers, refund customer
- Partial refund: May need to adjust transfers proportionally
- Transfer reversals: Use stripe.Transfer.create_reversal()
- Timing: Refunds can take 5-10 business days to appear


### 4.2 Transfer Reversals

Clawing back funds from a connected account:

In [76]:
# Reverse a transfer (partial or full)
reversal = stripe.Transfer.create_reversal(
    restaurant_transfer.id,
    amount=500,  # Reverse $5.00 (partial)
)

print("Transfer Reversal Use Cases:")
print("- Customer refund requiring funds back from connected account")
print("- Order cancellation before fulfillment")
print("- Dispute/chargeback recovery")
print("")
print("Important: Can only reverse if connected account has sufficient balance")

Transfer Reversal Use Cases:
- Customer refund requiring funds back from connected account
- Order cancellation before fulfillment
- Dispute/chargeback recovery

Important: Can only reverse if connected account has sufficient balance


### 4.3 Tips

In [77]:
# Option 1: Separate PaymentIntent for tip (recommended)
TIP_AMOUNT = 500  # $5.00 tip

print("Tip Handling Options:")
print("")
print("1. Separate PaymentIntent (Recommended)")
print("   - Create new PaymentIntent for tip amount")
print("   - Transfer 100% of tip to courier")
print("   - Clear audit trail, easy accounting")
print("")
print("2. Include in original order")
print("   - Customer adds tip before payment")
print("   - Adjust transfer amounts accordingly")
print("   - Simpler UX, but tip amount must be known upfront")

Tip Handling Options:

1. Separate PaymentIntent (Recommended)
   - Create new PaymentIntent for tip amount
   - Transfer 100% of tip to courier
   - Clear audit trail, easy accounting

2. Include in original order
   - Customer adds tip before payment
   - Adjust transfer amounts accordingly
   - Simpler UX, but tip amount must be known upfront


### 4.4 Instant Payouts for Couriers

Gig workers often want immediate access to earnings:

In [78]:
# Check if instant payouts are available
instant_payout = stripe.Payout.create(
    amount=700,
    currency="usd",
    method="instant",  # instant vs standard
    stripe_account=courier_account.id,
)

print("Payout Options:")
print("")
print("Standard Payout:")
print("  - 1-2 business days")
print("  - No additional fee")
print("")
print("Instant Payout:")
print("  - Minutes to debit card")
print("  - 1% fee (min $0.50)")
print("  - Requires eligible debit card")
print("  - Great for gig workers who need immediate access")

Payout Options:

Standard Payout:
  - 1-2 business days
  - No additional fee

Instant Payout:
  - Minutes to debit card
  - 1% fee (min $0.50)
  - Requires eligible debit card
  - Great for gig workers who need immediate access


---
# Cleanup

Clean up test data created during this demo:

In [79]:
# Cleanup: Delete all test objects created during this demo
# Run this cell to clean up your Stripe account

def safe_delete(delete_func, object_id, object_type):
    """Safely delete a Stripe object, handling errors gracefully."""
    try:
        if object_id and not object_id.startswith(('mock_', 'tr_mock', 'ch_mock', 'acct_mock')):
            delete_func(object_id)
            print(f"Deleted {object_type}: {object_id}")
            return True
        else:
            print(f"Skipped mock {object_type}: {object_id}")
            return False
    except stripe.InvalidRequestError as e:
        if "No such" in str(e):
            print(f"Skipped {object_type} already deleted or not found: {object_id}")
        else:
            print(f"Error deleting {object_type}: {e}")
        return False
    except Exception as e:
        print(f"Error deleting {object_type}: {e}")
        return False

print("=" * 50)
print("CLEANUP: Deleting Test Objects")
print("=" * 50)

deleted_count = 0

# Delete connected accounts (this will also remove any pending transfers)
print("\n--- Connected Accounts ---")
if 'restaurant_account' in dir():
    if safe_delete(lambda id: client.v1.accounts.delete(id), restaurant_account.id, "Restaurant Account"):
        deleted_count += 1
        
if 'courier_account' in dir():
    if safe_delete(lambda id: client.v1.accounts.delete(id), courier_account.id, "Courier Account"):
        deleted_count += 1

# Delete customer
print("\n--- Customers ---")
if 'customer' in dir():
    if safe_delete(lambda id: client.v1.customers.delete(id), customer.id, "Customer"):
        deleted_count += 1

# Note: PaymentIntents, Charges, and Transfers cannot be deleted
# They remain in your Stripe account for record-keeping
print("\n--- Payment Objects (Cannot be deleted) ---")
if 'payment_intent' in dir():
    print(f"PaymentIntent preserved: {payment_intent.id}")
if 'confirmed_payment' in dir() and hasattr(confirmed_payment, 'latest_charge'):
    charge_id = confirmed_payment.latest_charge
    if charge_id and not charge_id.startswith('ch_mock'):
        print(f"Charge preserved: {charge_id}")

print("\n" + "=" * 50)
print(f"Cleanup complete! Deleted {deleted_count} object(s).")
print("=" * 50)
print("\nNote: PaymentIntents, Charges, and Transfers are preserved")
print("in your Stripe Dashboard for record-keeping purposes.")

CLEANUP: Deleting Test Objects

--- Connected Accounts ---
Deleted Restaurant Account: acct_1SmfsmJCEva20iyf
Deleted Courier Account: acct_1SmftAJ5GsBgyCTt

--- Customers ---
Deleted Customer: cus_TkAETaIqXJ3Alb

--- Payment Objects (Cannot be deleted) ---
PaymentIntent preserved: pi_3SmftoJCEv6vweOQ0zQCUgWa
Charge preserved: ch_3SmftoJCEv6vweOQ039OQB5h

Cleanup complete! Deleted 3 object(s).

Note: PaymentIntents, Charges, and Transfers are preserved
in your Stripe Dashboard for record-keeping purposes.


---
# Key Takeaways

### Why Separate Charges & Transfers?

| Benefit | Description |
|---------|-------------|
| **Flexibility** | Control when and how much to transfer |
| **Multi-party** | Split payments to any number of recipients |
| **Timing** | Transfer after delivery confirmation, not before |
| **Disputes** | Platform holds funds, manages disputes centrally |

### Connect Account Types

| Feature | Custom | Express | Standard |
|---------|--------|---------|----------|
| Onboarding UI | Your own | Stripe hosted | Stripe hosted |
| Dashboard access | No | Limited | Full |
| Platform responsibility | High | Medium | Low |
| Best for | Marketplaces, delivery | Freelance platforms | SaaS, invoicing |